In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Upload the file manually
from google.colab import files
uploaded = files.upload()

# Load the uploaded CSV file
df = pd.read_csv("train.csv")  # replace with the actual file name


Saving train.csv to train.csv


In [2]:
# Load the CSV with bad line handling
df = pd.read_csv("train.csv", on_bad_lines='skip', encoding='utf-8')


In [3]:
# Check for missing values
print("Missing values per column:\n", df.isnull().sum())

# Decide on imputation or dropping based on data type and business logic.
# Example imputation (you may adjust this):
# Replace numerical columns' missing values with the mean or median
# Replace categorical columns with the mode
for column in df.columns:
    if df[column].isnull().sum() > 0:
        if df[column].dtype == 'object':  # Categorical column
            df[column].fillna(df[column].mode()[0], inplace=True)
        else:  # Numerical column
            df[column].fillna(df[column].mean(), inplace=True)

print("After imputation, missing values per column:\n", df.isnull().sum())


Missing values per column:
 Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64
After imputation, missing values per column:
 Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
New_Price            0
Price                0
dtype: int64


<ipython-input-3-ea9baaa9d8bc>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mode()[0], inplace=True)
<ipython-input-3-ea9baaa9d8bc>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

Justification: Since the mean maintains data integrity better than deleting rows, we utilized it for numerical data and the mode for categorical data.

In [4]:
import re

# Define a function to extract numeric values
def extract_numeric(value):
    try:
        return float(re.findall(r"[-+]?\d*\.\d+|\d+", str(value))[0])
    except IndexError:  # in case there is no numeric part found
        return np.nan  # or set a default value if appropriate

# Apply this function to relevant columns to clean the units
df['Mileage'] = df['Mileage'].apply(extract_numeric)
df['Engine'] = df['Engine'].apply(extract_numeric)
df['Power'] = df['Power'].apply(extract_numeric)
df['New_Price'] = df['New_Price'].apply(extract_numeric)

# Check the output
print(df[['Mileage', 'Engine', 'Power', 'New_Price']].head())


   Mileage  Engine   Power  New_Price
0    19.67  1582.0  126.20       4.78
1    13.00  1199.0   88.70       8.61
2    20.77  1248.0   88.76       4.78
3    15.20  1968.0  140.80       4.78
4    23.08  1461.0   63.10       4.78


In [5]:
# One-hot encode categorical variables
df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True)
print("Data after one-hot encoding:\n", df.head())


Data after one-hot encoding:
    Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven Owner_Type  Mileage  Engine   Power  Seats  New_Price  \
0              41000      First    19.67  1582.0  126.20    5.0       4.78   
1              46000      First    13.00  1199.0   88.70    5.0       8.61   
2              87000      First    20.77  1248.0   88.76    7.0       4.78   
3              40670     Second    15.20  1968.0  140.80    5.0       4.78   
4              86999      First    23.08  1461.0   63.10    5.0       4.78   

   Price  Fuel_Type_Electric  Fuel_Type_Petrol  Transmission_Manual  
0  12.50        

In [6]:
# Adding a new feature "Age"
df['Age'] = 2024 - df['Year']
print("Data after adding new feature:\n", df[['Year', 'Age']].head())


Data after adding new feature:
    Year  Age
0  2015    9
1  2011   13
2  2012   12
3  2013   11
4  2013   11


In [7]:
# Display column names to verify them
print("Column names in the dataset:", df.columns)


Column names in the dataset: Index(['Unnamed: 0', 'Name', 'Location', 'Year', 'Kilometers_Driven',
       'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats', 'New_Price',
       'Price', 'Fuel_Type_Electric', 'Fuel_Type_Petrol',
       'Transmission_Manual', 'Age'],
      dtype='object')


In [8]:
# Strip any leading/trailing whitespace from column names
df.columns = df.columns.str.strip()


In [9]:
# Select specific columns
selected_columns = df[['Name', 'Location', 'Price', 'Mileage', 'Engine']]
print("Selected columns:\n", selected_columns.head())

# Filter: Show cars with Mileage above 15 km/l
filtered_data = df[df['Mileage'] > 15]
print("Filtered data (Mileage > 15 km/l):\n", filtered_data.head())

# Rename: Rename the column "Price" to "Used_Price"
df.rename(columns={'Price': 'Used_Price'}, inplace=True)
print("Data with renamed column:\n", df.head())

# Mutate: Add a new feature, e.g., Price_per_Horsepower
df['Price_per_Horsepower'] = df['Used_Price'] / df['Power']
print("Data after adding Price_per_Horsepower:\n", df[['Used_Price', 'Power', 'Price_per_Horsepower']].head())

# Arrange: Sort by 'Used_Price'
arranged_data = df.sort_values(by='Used_Price', ascending=False)
print("Arranged data by Used_Price:\n", arranged_data.head())

# Summarize: Calculate the mean and max mileage grouped by Fuel Type (Petrol vs Electric)
summary = df.groupby('Fuel_Type_Petrol').agg({'Mileage': ['mean', 'max']})
print("Summary of Mileage grouped by Fuel Type:\n", summary)


Selected columns:
                                Name    Location  Price  Mileage  Engine
0  Hyundai Creta 1.6 CRDi SX Option        Pune  12.50    19.67  1582.0
1                      Honda Jazz V     Chennai   4.50    13.00  1199.0
2                 Maruti Ertiga VDI     Chennai   6.00    20.77  1248.0
3   Audi A4 New 2.0 TDI Multitronic  Coimbatore  17.74    15.20  1968.0
4            Nissan Micra Diesel XV      Jaipur   3.50    23.08  1461.0
Filtered data (Mileage > 15 km/l):
    Unnamed: 0                                 Name    Location  Year  \
0           1     Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
2           3                    Maruti Ertiga VDI     Chennai  2012   
3           4      Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6               Nissan Micra Diesel XV      Jaipur  2013   
6           8  Volkswagen Vento Diesel Comfortline        Pune  2013   

   Kilometers_Driven Owner_Type  Mileage  Engine   Power  Seats  New_Price  \
0 